# Debug Drill: The Leaky Pipeline

**Scenario:**
A colleague built a churn prediction model with impressive performance.

"AUC = 0.92!" they say. "Way better than our old model!"

But in production, the AUC drops to 0.72. "What happened?!"

**Your Task:**
1. Run the pipeline and spot the leakage
2. Identify TWO sources of leakage
3. Fix the pipeline properly
4. Write a 3-bullet postmortem

---

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

np.random.seed(42)

In [ ]:
# Generate customer data
n = 2000

df = pd.DataFrame({
    'customer_id': range(n),
    'tenure_days': np.random.uniform(30, 1000, n),
    'monthly_spend': np.random.exponential(100, n),
    'support_tickets': np.random.poisson(2, n),
    'logins_last_30d': np.random.poisson(15, n),
})

# Generate churn based on features
churn_prob = 1 / (1 + np.exp(
    2 - 0.002 * df['tenure_days'] - 0.005 * df['monthly_spend'] +
    0.2 * df['support_tickets'] - 0.05 * df['logins_last_30d']
))
df['churn'] = (np.random.random(n) < churn_prob).astype(int)

print(f"Dataset: {len(df)} customers")
print(f"Churn rate: {df['churn'].mean():.1%}")

In [ ]:
# ===== COLLEAGUE'S CODE (TWO BUGS!) =====

# Features
feature_cols = ['tenure_days', 'monthly_spend', 'support_tickets', 'logins_last_30d']
X = df[feature_cols].copy()
y = df['churn']

# BUG 1: Scale BEFORE splitting (leaks test statistics into train)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # <-- Fitted on ALL data!

# BUG 2: Create a target-derived feature (leaks the label itself)
# "Average churn rate for similar customers" - sounds reasonable, right?
df['segment'] = pd.qcut(df['tenure_days'], q=5, labels=['New', 'Active', 'Established', 'Loyal', 'Veteran'])
segment_churn_rate = df.groupby('segment')['churn'].transform('mean')  # <-- Uses ALL churn labels!
X_scaled = np.column_stack([X_scaled, segment_churn_rate])

# Now split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)

# Train model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Evaluate
train_auc = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
test_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

print("=== Colleague's Results ===")
print(f"Train AUC: {train_auc:.3f}")
print(f"Test AUC:  {test_auc:.3f}")
print("\n🎉 Amazing performance! Deploying...")
print("\n⚠️ But in production, AUC drops to ~0.72. Why?")

---

## Your Investigation

### Step 1: Identify the leakage sources

In [ ]:
print("=== Leakage Analysis ===")
print()
print("❌ BUG 1: Scaler fit on ALL data")
print("   - StandardScaler.fit_transform() called BEFORE train_test_split")
print("   - Test set statistics leak into the scaled training data")
print("   - Result: Optimistic performance estimate")
print()
print("❌ BUG 2: Target-derived feature")
print("   - 'segment_churn_rate' is calculated using ALL churn labels")
print("   - This includes test set labels!")
print("   - The feature directly encodes the target")
print()
print("💡 Production reality:")
print("   - Can't use future data to scale")
print("   - Can't use the churn label you're trying to predict")

In [ ]:
# Show how much the leaky feature helps
print("=== Feature Importance (Leaky Model) ===")
feature_names = feature_cols + ['segment_churn_rate']
for name, coef in zip(feature_names, model.coef_[0]):
    flag = "⚠️ LEAKY" if name == 'segment_churn_rate' else ""
    print(f"  {name:<20}: {coef:>7.3f} {flag}")

print("\n🔍 The leaky feature has the highest coefficient!")

### Step 2: TODO - Fix the pipeline

In [ ]:
# TODO: Build a leak-free pipeline

# Uncomment and complete:

# # Step 1: Use only safe features (no target-derived)
# X_clean = df[feature_cols].copy()
# y_clean = df['churn']
# 
# # Step 2: Split FIRST
# X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(
#     X_clean, y_clean, test_size=0.3, random_state=42, stratify=y_clean
# )
# 
# # Step 3: Fit scaler on TRAIN only
# scaler_clean = StandardScaler()
# X_train_scaled = scaler_clean.fit_transform(X_train_clean)  # Fit on train
# X_test_scaled = scaler_clean.transform(X_test_clean)         # Transform test (no fit!)
# 
# # Step 4: Train model
# model_clean = LogisticRegression(max_iter=1000)
# model_clean.fit(X_train_scaled, y_train_clean)
# 
# # Step 5: Evaluate
# train_auc_clean = roc_auc_score(y_train_clean, model_clean.predict_proba(X_train_scaled)[:, 1])
# test_auc_clean = roc_auc_score(y_test_clean, model_clean.predict_proba(X_test_scaled)[:, 1])
# 
# print("=== Clean Pipeline Results ===")
# print(f"Train AUC: {train_auc_clean:.3f}")
# print(f"Test AUC:  {test_auc_clean:.3f}")

In [ ]:
# TODO: Compare leaky vs clean

# Uncomment:

# print("=== Comparison ===")
# print(f"\n                    Leaky Pipeline    Clean Pipeline")
# print(f"  Train AUC:        {train_auc:>14.3f}    {train_auc_clean:>14.3f}")
# print(f"  Test AUC:         {test_auc:>14.3f}    {test_auc_clean:>14.3f}")
# print(f"\n  Leakage inflation: {test_auc - test_auc_clean:+.3f}")
# print(f"\n💡 The 'clean' test AUC is what you'll actually get in production!")

In [ ]:
# ============================================
# SELF-CHECK
# ============================================

# Uncomment:

# assert test_auc_clean < test_auc, "Clean pipeline should have lower (realistic) AUC"
# assert X_train_scaled.shape[1] == len(feature_cols), "Should have only safe features"
# 
# print("✓ Pipeline fixed!")
# print(f"✓ Removed leaky feature")
# print(f"✓ Scaler fitted on train only")
# print(f"✓ Realistic AUC: {test_auc_clean:.3f}")

### Step 3: Write your postmortem

In [ ]:
postmortem = """
## Postmortem: The Leaky Pipeline

### What happened:
- (Your answer: What was the gap between test and production performance?)

### Root cause:
- (Your answer: Name the two leakage sources)

### How to prevent:
- (Your answer: What's the correct order of operations?)

"""

print(postmortem)

---

## ✅ Drill Complete!

**Key lessons:**

1. **Always split first.** Any transformation that uses statistics (mean, std, target encoding) must be fit on train only.

2. **Watch for target-derived features.** If a feature uses the label in its calculation, it's leaking.

3. **The correct order:** Split → Fit transforms on train → Apply to train and test

4. **If AUC seems too good to be true, it probably is.**

---

## Common Leakage Sources

| Source | Example | Fix |
|--------|---------|-----|
| Scaling before split | `fit_transform(X)` then split | Split first, fit on train |
| Target encoding on all data | `groupby().transform('mean')` | Use CV encoding |
| Future features | `next_month_orders` | Check temporal logic |
| Features derived from target | `segment_churn_rate` | Remove or use proper encoding |